In [ ]:

import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp
from genpen.utils import Paper
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from skimage import exposure

from skimage import filters
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.morphology import disk
from pyaxidraw import axidraw   # import module

from PIL import Image

import fn
import cv2

from genpen.flow.field import *
from genpen.flow.particle import *
from genpen.genpen import random_line_subdivide_gen

import time

%load_ext autoreload
%autoreload 2

# more complex

In [ ]:

paper_size:str = '6x5.99 inches'
border:float=15  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)


In [ ]:
center = drawbox.centroid

In [ ]:
db = gp.Shape(drawbox)

In [ ]:
drawbox.bounds

In [ ]:
corner = Point(35,center.y)

In [ ]:
n_circles = 3
xcenter = 100
yrange = np.linspace(db.bottom+30, db.top-30, n_circles)

In [ ]:
pts = [Point(xcenter, y) for y in yrange]

In [ ]:
radmin = 20
radmax = 20
rads = np.linspace(radmin, radmax, len(pts))
circles = [pt.buffer(d) for d,pt in zip(rads, pts)]

In [ ]:
prms = gp.ScaleTransPrms(n_iters=300)

In [ ]:
cs = [gp.Poly(c) for c in circles]

In [ ]:
for c in cs:
    c.fill_hatch(angle=0, spacing=0.25)

In [ ]:
fillsets

In [ ]:
fillsets = [c.fill for c in cs]
shotsets = []
for fills in fillsets:
    gun_pts = [corner] * len(fills)
    substring_start_ds = ss.uniform(loc=0.05, scale=0.25).rvs
    substring_len_ds = ss.uniform(loc=0.4, scale=0.3).rvs
    shots = shoot_it(
        linestrings=fills, 
        gun_pts=gun_pts, 
        substring_start_ds=substring_start_ds, 
        substring_len_ds=substring_len_ds)
    shotsets.append(shots)

In [ ]:
def shoot_it(linestrings, gun_pts, substring_start_ds, substring_len_ds):
    substrings = []
    for ii, ls in enumerate(linestrings):
        endpt = ls.interpolate(np.random.uniform(), normalized=True)
        connection = LineString((gun_pts[ii], endpt))
        substring_start_d = substring_start_ds()
        substring_end_d = substring_start_d + substring_len_ds()
        substring_start = connection.interpolate(substring_start_d, normalized=True)
        substring_end = connection.interpolate(substring_end_d, normalized=True)
        substring = LineString((substring_start, substring_end))
        substrings.append(substring)
    substrings = gp.merge_LineStrings(substrings)
    return substrings

In [ ]:


   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
counter = 1

for shots, fills in zip(shotsets, fillsets):
    for s,f, in zip(shots, fills):
        sk.stroke(counter)
        sk.geometry(s)
        counter += 1
        sk.stroke(counter)
        sk.geometry(f)
        counter += 1
    
    
# tolerance=0.5
sk.display()

In [ ]:
filename = fn.Fn(postfix='-machine_gun.svg', git_sha_size=5, pid_sha_size=6).name(utc=False, milli=False)

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)